# 顔抽出作業notebook
ここで試したものをスクリプト化したものが `extract_face.py`  
このときはまだデータセットの構成が定まっていなかった

ページ単位での顔抽出とコマ単位での顔抽出を試したけど、結局コマ単位でやることにした

# TODO

- [x] newgame以外でも動かせるように汎用化する
- [x] notebookからスクリプトに直す

In [1]:
# https://github.com/nagadomi/lbpcascade_animeface
!wget https://raw.githubusercontent.com/nagadomi/lbpcascade_animeface/master/lbpcascade_animeface.xml

--2018-04-16 23:39:23--  https://raw.githubusercontent.com/nagadomi/lbpcascade_animeface/master/lbpcascade_animeface.xml
raw.githubusercontent.com (raw.githubusercontent.com) をDNSに問いあわせています... 151.101.72.133
raw.githubusercontent.com (raw.githubusercontent.com)|151.101.72.133|:443 に接続しています... 接続しました。
HTTP による接続要求を送信しました、応答を待っています... 200 OK
長さ: 59157 (58K) [text/plain]
`lbpcascade_animeface.xml' に保存中

lbpcascade_animefac 100%[===================>]  57.77K  --.-KB/s 時間 0.02s      

2018-04-16 23:39:23 (2.60 MB/s) - `lbpcascade_animeface.xml' へ保存完了 [59157/59157]



In [2]:
import cv2
# in: original image
# out: detect image
def detect(image, cascade_file = "lbpcascade_animeface.xml"):
    cascade = cv2.CascadeClassifier(cascade_file)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.equalizeHist(gray)
    faces = cascade.detectMultiScale(gray,
                                     # detector options
                                     scaleFactor = 1.1,
                                     minNeighbors = 5,
                                     minSize = (24, 24))
    for (x, y, w, h) in faces:
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 0, 255), 2)
    #img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # debug
    #plt.imshow(img)                                                            # debug
    #cv2.imwrite("out.png", image)
    return image

In [ ]:
# page単位認識
input_base = '/Users/ragi/workspace/4koma_project/4koma/newgame/original/'
output_base = '/Users/ragi/workspace/4koma_project/4koma/newgame/face/'

last_page = [None, 117, 118, 117, 117, 118]

for comic_num in range(1, 5+1):
    comic_path = os.path.join(input_base, f'newgame{comic_num}/')
    for page_file in os.listdir(comic_path):
        if not page_file[0].isdigit():
            continue
        input_path = os.path.join(comic_path, page_file)
        image = cv2.imread(input_path, cv2.IMREAD_COLOR)
        detect_image = detect(image)
        output_path = os.path.join(output_base, f'newgame{comic_num}', page_file)
        cv2.imwrite(output_path, image)

In [ ]:
# コマ単位認識
input_base = '/Users/ragi/workspace/4koma_project/4koma/newgame/frame/'
output_base = '/Users/ragi/workspace/4koma_project/4koma/newgame/face/'

for comic_num in range(1, 5+1):
    comic_path = os.path.join(input_base, f'newgame{comic_num}/')
    for page_num in os.listdir(comic_path):
        if not page_num.isdigit():
            continue
        page_path = os.path.join(comic_path, page_num)
        for frame_file in os.listdir(page_path):
            frame_path = os.path.join(page_path, frame_file)
            image = cv2.imread(frame_path, cv2.IMREAD_COLOR)
            detect_image = detect(image)
            output_dir = os.path.join(output_base, f'frame_face_newgame{comic_num}', page_num)
            os.makedirs(output_dir, exist_ok=True)
            output_path = os.path.join(output_dir, frame_file)
            cv2.imwrite(output_path, image)

In [ ]:
# 顔抽出
input_base = '/Users/ragi/workspace/notebooks/output'
output_base = '/Users/ragi/workspace/notebooks/output/'
cascade = cv2.CascadeClassifier('lbpcascade_animeface.xml')

for comic_num in range(1, 5+1):
    face_id = 0
    comic_path = os.path.join(input_base, f'newgame{comic_num}/')
    output_dir = os.path.join(output_base, f'face_newgame{comic_num}/')
    for page_num in os.listdir(comic_path):
        if not page_num.isdigit():
            continue
        page_path = os.path.join(comic_path, page_num)
        for frame_file in os.listdir(page_path):
            frame_path = os.path.join(page_path, frame_file)
            image = cv2.imread(frame_path, cv2.IMREAD_COLOR)

            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            gray = cv2.equalizeHist(gray)
            faces = cascade.detectMultiScale(gray,
                                             # detector options
                                             scaleFactor = 1.1,
                                             minNeighbors = 5,
                                             minSize = (24, 24))
            for (x, y, w, h) in faces:
                face_image = image[y:y+h, x:x+w]
                output_path = os.path.join(output_dir, f'{face_id}.jpg')
                cv2.imwrite(output_path, face_image)
                face_id += 1